In [1]:
import os
import requests
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

dataset_path = './dataset/'

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
else:
    print(f"The folder '{dataset_path}' already exists.")

In [3]:
def save_image(url, file_name):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.5',
        }
        try:
            response = requests.get(url, headers=headers, timeout=10, verify=False)
            response.raise_for_status()
        except requests.exceptions.SSLError:
            print(f"SSL error occurred for the domain {file_name}.")
        except requests.exceptions.RequestException as e:
            print(f"Error accessing the domain {file_name}: {e}")
            return None
        
        url_path = urlparse(url).path
        ext = os.path.splitext(url_path)[1]
        if not ext:
            ext = '.jpeg'
        
        file_path = os.path.join(dataset_path, file_name + ext)
        with open(file_path, 'wb') as f:
            f.write(response.content)
    except Exception as e:
        print(f"Error downloading the image from the domain {file_name}: {e}")


In [4]:
def get_favicon_from_html(domain):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
            'Referer': f'https://www.{domain}/',
            'Accept-Language': 'en-US,en;q=0.5'
        }

        try:
            response = requests.get(f'https://www.{domain}', verify=False, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.SSLError:
            response = requests.get(f'http://www.{domain}', verify=False, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            return str(e), False

        soup = BeautifulSoup(response.text, 'html.parser')

        favicon_url = None
        link_tag = soup.find('link', rel='icon')
        if link_tag:
            favicon_url = link_tag.get('href')

        if not favicon_url:
            link_tag = soup.find('link', rel='shortcut icon')
            if link_tag:
                favicon_url = link_tag.get('href')

        if not favicon_url:
            favicon_url = '/favicon.ico'

        return favicon_url, True
    except requests.exceptions.RequestException as e:
        return str(e), False

In [ ]:
def scrape_favicons_from_domains(domains):
    for index, domain in enumerate(domains):
        file_name = domain + '_' + str(index)
        try:
            favicon_url, success = get_favicon_from_html(domain)
            if success:
                favicon_url = urljoin(f'https://www.{domain}', favicon_url)
                save_image(favicon_url, file_name)
            else:
                print(f"Favicon not found for the domain {file_name}: {favicon_url}")
        except Exception as e:
            print(f"Error processing domain {file_name}: {e}")

In [6]:
df = pd.read_parquet("logos.snappy.parquet", engine="pyarrow")

domains = df['domain']

scrape_favicons_from_domains(domains)

Error accessing the domain ccusa.co.nz_11: 404 Client Error: Not Found for url: https://ccusa.co.nz/wp-content/uploads/2023/02/elementor/thumbs/favicon_noGlobeRound-qajrjo512ai2oh7ixchhx7bnlqb2o6o9g5aqgvfk38.png
Favicon not found for the domain zalando.cz_13
Error accessing the domain avis.cr_22: 404 Client Error: Not Found for url: https://avis.cr/favico.ico
Favicon not found for the domain oil-testing.de_34
Favicon not found for the domain xella-colloquium.berlin_45
Error accessing the domain trinkgutlippstadt.de_53: HTTPSConnectionPool(host='trinkgutgeilenkirchen.de', port=443): Max retries exceeded with url: /wp-content/uploads/2014/02/favicon.png (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C39B1FF890>: Failed to resolve 'trinkgutgeilenkirchen.de' ([Errno 11001] getaddrinfo failed)"))
Favicon not found for the domain avoncameroun.com_57
Favicon not found for the domain svnefird.com_61
Favicon not found for the domain noiseassessments.org_64

KeyboardInterrupt: 